### Prediction of the best 100 molecules in the untested dataset
in findBestModel.ipynb is found thart model RandomForestClassifier_PCA1_std.pkl showed the best test score of all tried settings in the trained models. 

In [1]:
import pandas as pd
import MachineLearning  as ml
import joblib
from rdkit import Chem, DataStructs, RDLogger
from rdkit.Chem import Draw, Descriptors, AllChem

In [2]:
# import untested dataset 
molecules = ml.importFiles(nrFiles=1)

ml.writeNewMolfile(molecules)
allDescrs = ml.createDescriptorDf()
x = ml.generateMorganFingerprint()
# there is no target
# y = molecules[1][1:].astype(int)

x, xDims = ml.CombineDescriptorsAndFigerprints(x,allDescrs)

xScale = ml.scaleData(x,scaletype='standardize')

2023-06-18 17:16:35.105 python[29807:1285251] +[CATransaction synchronize] called within transaction
2023-06-18 17:16:35.419 python[29807:1285251] +[CATransaction synchronize] called within transaction


All molecules stored in: AllTestedMols.txt


In [10]:
# xScale.head()
q = xScale.iloc[:4]

In [6]:
# save since previous step takes a lot of time
import numpy as np
np.save('ScaledDescriporData.npy', xScale)

In [17]:
smiles = xScale["SMILES"]
xScaleNoSMILES = xScale.drop(columns=["SMILES"])

In [18]:
trainedModel = joblib.load("RandomForestClassifier_PCA1_std.pkl")
predProb = trainedModel.predict_proba(xScaleNoSMILES)
predProbDf = pd.DataFrame(predProb)
predProbDf.insert(loc=0, column='SMILES', value=smiles)

In [22]:
predProbDf.iloc[:4,:]

,SMILES,0,1
0,CCCCC/C(=N\NC(=O)CCC(=O)NCc1ccccc1)c1ccccc1,0.38,0.62
1,Cc1ccc(S(=O)(=O)N2CCC(C(=O)OCC(=O)N3CCc4ccccc4...,0.27,0.73
2,COc1ccc(N2C(=O)CC(Nc3ccc(F)cc3)C2=O)c([N+](=O)...,0.44,0.56
3,Cc1cc(C)cc(OCC(=O)O[NH+]=C(N)Cc2ccc([N+](=O)[O...,0.56,0.44


In [27]:
predProbDfSort = predProbDf.sort_values(by=1, ascending=False)
top100Mols = predProbDfSort.head(100)

In [4]:
import MachineLearning  as ml
top100MolsWithProbs = ml.top100molecules('RandomForestClassifier_PCA999_std.pkl', 0.999)
top100Mols = top100MolsWithProbs['SMILES']
top100Mols.to_csv('top100UntestedMoleculesT.csv', index=False)

2023-06-19 22:54:21.799 python[49104:2235406] +[CATransaction synchronize] called within transaction
2023-06-19 22:54:21.945 python[49104:2235406] +[CATransaction synchronize] called within transaction
2023-06-19 22:54:22.244 python[49104:2235406] +[CATransaction synchronize] called within transaction
